In [1]:
# this scripts purpose is for the extraction of the e's from a given pdf and 
# being stored in a path provided

# we have made a dir -> saved_e, this stores the e's of concern

In [23]:
import cv2
import pytesseract
import numpy as np
from PIL import Image, ImageDraw
import os

In [132]:
# Load the image
image_path = 'test_images/test_0.png'  # Replace with your image
img1 = cv2.imread(image_path)
img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

#upscalling images, solely to help tesseract, image extraction will be done from the og image
# img = cv2.resize(img2, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
img = cv2.resize(img2, (img1.shape[1], img1.shape[0]), interpolation=cv2.INTER_CUBIC)   # this is the processed image which 
                                                                                       # we put in the tesseract
                                                                                       # coz it needs clearer pcitures to 
                                                                                       # accurately locate the e's
# print(img.shape, img2.shape)

# Get image height (Tesseract origin is bottom-left)
h, w = img.shape

op_dir = 'saved_e'
os.makedirs(op_dir, exist_ok=True)

# Draw setup
# draw = ImageDraw.Draw(img)

# Get character bounding boxes
boxes = pytesseract.image_to_boxes(img)
count = 0

#set dimensions of the boxes 
crop = 14
half = crop//2

# Draw green dots on top of each 'e'
for b in boxes.strip().splitlines():
    b = b.split()
    char, x1, y1, x2, y2 = b[0], int(b[1]), int(b[2]), int(b[3]), int(b[4])
    
    # Flip y-coordinates
    y1_new = h - y1
    y2_new = h - y2
    # cx = (x1 + x2) // 2
    # cy = (y1_new + y2_new) // 2

    # make a bounding box around the e
    cv2.rectangle(img, (x1, y2_new), (x2, y1_new), (255, 0, 0), 1)

    if char.lower() == 'e' and count<10:

        # compute center of bounding box    
        cx = (x1 + x2) // 2
        cy = (y1_new + y2_new) // 2
    
        # Get top-left and bottom-right coordinates of fixed crop
        x_start = max(0, cx - half)
        y_start = max(0, cy - half)
        x_end = min(img2.shape[1], cx + half)
        y_end = min(img2.shape[0], cy + half)

        # crop image
        cropped = img2[y_start:y_end, x_start:x_end]

        # if dimensions okay, at image to output dir, 
        # later make sure that the image dirs are constant, else will cause computation error
        if cropped.shape[0] > 0 and cropped.shape[1] > 0:
            out_path = os.path.join(op_dir, f"e_{count+1}.png")
            cv2.imwrite(out_path, cropped)
            count += 1

cv2.imwrite("output_with_boxes.png", img)
print(f"Saved {count} 'e' characters in '{op_dir}/' and annotated image as 'output_with_boxes.png'")

# # to check the dimensions of the images
# for filename in os.listdir(op_dir):
#     full_path = os.path.join(op_dir, filename)
#     img_saved = cv2.imread(full_path)
    
#     if img_saved is not None:
#         print(f"{filename}: {img_saved.shape}")
#     else:
#         print(f"Failed to read {filename}")

Saved 10 'e' characters in 'saved_e/' and annotated image as 'output_with_boxes.png'
e_5.png: (14, 14, 3)
e_10.png: (14, 14, 3)
e_1.png: (14, 14, 3)
e_3.png: (14, 14, 3)
e_6.png: (14, 14, 3)
e_9.png: (14, 14, 3)
e_7.png: (14, 14, 3)
e_2.png: (14, 14, 3)
e_8.png: (14, 14, 3)
e_4.png: (14, 14, 3)
